### Sample SQL Queries

Count the number of patients per sign:
```
SELECT patients.sign, COUNT(id) FROM patients GROUP BY patients.sign

```
Select the first variants gene for each patient
```
select report->'variants'->1 from reports
```

Select all Libras between 21 and 40 with BRCA1 and PTEN
```
select patients.id, report->'variants' 
from patients
left join reports on (patients.id = reports.patient_id)
where patients.sign='Libra' and patients.age >= 21 and patients.age <= 40
and report @> '{ "variants": [{ "gene": "BRCA1" }] }' and report @> '{ "variants": [{ "gene": "PTEN" }] }'

```

In [48]:
import json
from contextlib import closing
import psycopg2
import psycopg2.extras

In [20]:
# Connect to the postgres server linked to the container this notebook is running in
connection = psycopg2.connect("host=postgres dbname=wisdom user=postgres")

In [11]:
cursor.close()

In [19]:
connection.close()

In [15]:
# Query the first variant from every report and print its gene
with closing(connection.cursor()) as cursor:
    cursor.execute("select report->'variants'->1 from reports")
    variants = cursor.fetchall()
    print("Found variants for", len(variants), "patients")
    print([v[0]["gene"] for v in variants])

Found variants for 250 patients
['MSH2', 'ATM', 'ATM', 'MSH6', 'BRCA2', 'PALB2', 'MLH1', 'PALB2', 'BRCA1', 'ATM', 'ATM', 'MLH1', 'ATM', 'ATM', 'ATM', 'BRCA1', 'ATM', 'ATM', 'BRIP1', 'BRIP1', 'MSH6', 'ATM', 'PMS2', 'ATM', 'BRCA2', 'ATM', 'ATM', 'ATM', 'MSH2', 'PALB2', 'PALB2', 'BRCA1', 'PTEN', 'BRCA1', 'PALB2', 'BRCA1', 'CDH1', 'ATM', 'MSH6', 'BRCA1', 'ATM', 'ATM', 'BRIP1', 'BRCA1', 'ATM', 'BRCA2', 'PTEN', 'BRCA1', 'BRCA2', 'ATM', 'ATM', 'BRCA1', 'ATM', 'ATM', 'BRCA1', 'CDH1', 'BRCA1', 'BRIP1', 'BRCA1', 'ATM', 'BRCA1', 'ATM', 'BRCA1', 'PALB2', 'BRIP1', 'MSH2', 'BRCA2', 'BRCA1', 'PTEN', 'ATM', 'MSH6', 'BRCA1', 'BRCA1', 'ATM', 'ATM', 'PALB2', 'BRIP1', 'MSH2', 'PALB2', 'BRCA2', 'ATM', 'CDH1', 'PALB2', 'MLH1', 'ATM', 'BRCA2', 'ATM', 'ATM', 'PTEN', 'ATM', 'CDH1', 'BRCA2', 'ATM', 'BRCA1', 'ATM', 'MLH1', 'PTEN', 'MSH2', 'BRCA2', 'BRCA2', 'BRCA1', 'PALB2', 'BRCA2', 'MLH1', 'MSH6', 'ATM', 'BRCA1', 'BRCA1', 'MSH2', 'PMS2', 'MLH1', 'BRCA1', 'BRCA2', 'ATM', 'MSH6', 'BRCA1', 'PTEN', 'ATM', 'BRCA1', 

In [82]:
# Find all Libras between 21 and 40 with BRCA1 and PTEN
with closing(psycopg2.connect("host=postgres dbname=wisdom  user=postgres")) as connection, \
    closing(connection.cursor(cursor_factory=psycopg2.extras.DictCursor)) as cursor:
    cursor.execute("select * from patients where sign='Libra' and age >= 21 and age <= 40")
    patients = cursor.fetchall()
    print("Found", len(patients), "Libra's between 21 and 40")
    
    print("Those with both BRCA1 and PTEN:")
    for patient in patients:
        cursor.execute("select report->'variants' from reports where reports.patient_id='{}'".format(patient["id"]))
        variants = cursor.fetchall()
        genes = [v["gene"] for v in variants[0][0]]
        if 'BRCA1' in genes and 'PTEN' in genes:
            print(patient["id"], genes)

Found 11 Libra's between 21 and 40
Those with both BRCA1 and PTEN:
5da839ef-2c57-4457-a043-aa3b877021d0 ['PALB2', 'ATM', 'BRCA1', 'BRCA1', 'PMS2', 'BRCA2', 'PTEN', 'ATM']
82a8a1b6-1b90-4eb7-af7d-e3304c8c2339 ['MLH1', 'PTEN', 'ATM', 'CDH1', 'BRCA2', 'BRCA1', 'PALB2', 'ATM']
875cd09a-2e0e-4ee6-a70c-efbef489e77a ['BRIP1', 'BRCA2', 'PTEN', 'CDH1', 'PALB2', 'ATM', 'MSH2', 'BRCA1']
e7a547de-c199-44c8-8b25-31ad0102f31c ['BRCA1', 'MLH1', 'PTEN', 'BRCA1', 'BRCA2', 'ATM', 'BRIP1', 'ATM']
d2a0f816-127d-48af-9d52-2be7dc480ae7 ['PTEN', 'BRCA1', 'MSH6', 'PMS2', 'BRCA1', 'ATM', 'PALB2', 'ATM']
6e9855f8-a0b3-4ff5-9761-758f108fd9ec ['ATM', 'BRCA1', 'BRIP1', 'PALB2', 'MLH1', 'PALB2', 'MSH2', 'PTEN']
